In [1]:
from vocab import Vocab
import utils
from sc_model import SC
import torch.nn as nn
import torch 
from nltk import word_tokenize

In [2]:
# VocabEntry = Vocab.from_corpus("data_raw.txt", 50000, freq_cutoff=5)

In [3]:
# model = SC(num_layers = 4, d_model =512, nhead = 16, hidden_dim = 256 , vocab = VocabEntry)

In [3]:
# model.save("model.bin")
model = SC.load("model1.bin").cuda()

In [4]:
x = "những người biểu tình đập phá trạm xe điện tạy sa điền và một khátr sạn bị cho là liên quan tới bắk kinh "
x = [word_tokenize(x)]
def check(x):
    y = model(x)
    y = torch.round(y[0])
    print("Từ bị sai :")
    for i in range(len(y)):
        if y[i] == 1:
            print(x[0][i])
check(x)

Từ bị sai :
tạy
khátr
bắk


In [2]:
data_train = []
# k= 0
for line in open("data_train.txt", encoding="utf8"):
    data_train.append(line.strip().lower())
  # k+= 1
  # if k == 30000:
  #   break

In [ ]:
# train
# f = open('30000_data.txt', 'w')

# for i in range(30000):
#     f.write(data_train[i]+ "\n")
# f.close()

In [ ]:
data_label = []
for line in open("data_label.txt"):
    data_label.append(line.strip().lower())
  # if k == 30000:
  #   break

In [ ]:
data_dev = []
k= 0
for line in open("data_dev.txt", encoding="utf8"):
    k += 1
    data_dev.append(word_tokenize(line.strip().lower()))
    if k ==1000:
        break


In [ ]:
k= 0
data_dev_label = []
for line in open("data_dev_label.txt", encoding="utf8"):
    k += 1
    data_dev_label.append(line.strip().lower())
    if k ==1000:
        break

In [ ]:
dev_label = [word_tokenize(seq) for seq in data_dev_label]
max_l = 0
for seg in dev_label:
    if len(seg) > max_l:
        max_l = len(seg)
for i in range(len(dev_label)):
    dev_label[i] = dev_label[i] + (max_l - len(dev_label[i]))*['0']
    for j in range(len(dev_label[i])):
        dev_label[i][j] = int(dev_label[i][j])
dev_label = torch.Tensor(dev_label).cuda()

In [ ]:
from torch.utils.data import DataLoader
def length(data):
    return torch.LongTensor([len(seq) for seq in data])
batch_size = 64
def collate_fn(batch):
    data,label = zip(*batch)
    y = [word_tokenize(seq) for seq in label]
    max_l = 0
    for seg in y:
        if len(seg) > max_l:
            max_l = len(seg)
    for i in range(len(y)):
        y[i] = y[i] + (max_l - len(y[i]))*['0']
        for j in range(len(y[i])):
            y[i][j] = int(y[i][j])
    x = [word_tokenize(seq) for seq in data]
    return x, torch.Tensor(y).cuda()

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lossfunction = nn.BCELoss(reduction = "sum")

In [ ]:
lr_decay = 0.5
epochs = 50
model_save_path = "model.bin"
patience = 0

In [ ]:
data = list(zip(data_train,data_label))
loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last = True )
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in loader:
        
        optimizer.zero_grad()
        
        y_pred, lengths = model(x) 
        
        loss = lossfunction(y_pred, y)/sum(lengths)
        
        loss.backward()
        optimizer.step()
        total_loss += loss
    
    # Early stop and decay lr
    if epoch %2 ==0:
        print("Loss: {:.2f}, epoch: {}".format(total_loss, epoch))
        model.eval()
        with torch.no_grad():
            y_dev_pred, length_dev = model(data_dev)
            score = lossfunction(y_dev_pred, dev_label)/sum(length_dev)
        if epoch == 0:
            best_score = score
            print("Save model with dev_f1_score = {:.2f}".format(best_score))
            model.save(model_save_path)
        if score< best_score:
            patience = 0
            best_score = score
            print("Save model with dev_f1_score = {:.2f}".format(best_score))
            model.save(model_save_path)
            
        else:
            patience += 1
        if patience == 3:
            # decay lr, and restore from previously best checkpoint
            lr = optimizer.param_groups[0]['lr'] * lr_decay
            print('Load previously best model and decay learning rate to %f' % lr)
            model = SC.load(model_save_path).cuda()
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            
            # reset patience
            patience = 0

In [ ]:
x = torch.Tensor([2,3])
y = [2,3]
x/sum(y)